Packages

In [13]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import functools
import operator
import pandas as pd
import netCDF4 as nc
import pandas as pd
import xarray as xr
import os

Parameters

In [14]:
Horizon = 2040
start_year = 2025
LeapYearsInvestment = 5
total_periods = int((Horizon - start_year)/LeapYearsInvestment)
usd_to_eur = 0.9188
pypsa_tech = load_workbook('data_pypsa/technologies.xlsx').active
bus_map = load_workbook('data_pypsa/bus_map30.xlsx').active
new_names = [cell.value for cell in bus_map['A'][1:]]
scale_demand = {
    2020: 1.0,
    2021: 1.0,
    2022: 1.0,
    2023: 1.0,
    2024: 1.0,
    2025: 1.060816836,
    2026: 1.033372562,
    2027: 1.027658017,
    2028: 1.032454682,
    2029: 1.030974469,
    2030: 1.030564438,
    2031: 1.029585799,
    2032: 1.029366414,
    2033: 1.048001634,
    2034: 1.041580000,
    2035: 1.041580000,
    2036: 1.041580000,
    2037: 1.041580000,
    2038: 1.041580000,
    2039: 1.041580000,
    2040: 1.041580000,
    2041: 1.041580000,
    2042: 1.041580000,
    2043: 1.041580000,
    2044: 1.041580000,
    2045: 1.041580000,
    2046: 1.041580000,
    2047: 1.041580000,
    2048: 1.041580000,
    2049: 1.041580000,
    2050: 1.041580000,
}
pathway = 'base_NZE'
generator_tech_dict = {'Gas OCGT': ['Gas'], 'Gas CCGT': ['Gas'], 'Oil existing': ['Existing', 'Oil'], 'Hydro regulated': ['Hydro_reg'], 'Hydro run-of-the-river': ['Hydro_ror'], 'Bio': ['Bio'], 'Geo': ['Geo'], 'Wind onshore': ['Wind_onshr'], 'Solar': ['Solar']}
generators = pd.read_excel('data_pypsa/generators30.xlsx')
storage_units = pd.read_excel('data_pypsa/storage_units30.xlsx')
val = True

In [15]:
# Original decom dictionary

decom = {
    2025: {'BO0 1': ['OCGT', 16.84]},
    2026: {'BO0 5': ['OCGT', 49.76 + 51.37]},
    2027: {'BO0 1': ['OCGT', 15.99]},
    2028: {},
    2029: {'BO0 1': ['OCGT', 57.14 + 55.97]},
    2030: {'BO0 1': ['OCGT', 18.1], 'BO0 5': ['OCGT', 42.41 + 41.15]},
    2031: {'BO0 18': ['hydro', 2.55]},
    2032: {'BO0 10': ['oil', 1.1 + 1.12 + 1.12]},
    2033: {},
    2034: {},
    2035: {'BO0 10': ['oil', 1.1], 'BO0 28': ['oil', 1.1], 'BO0 2': ['solar', 5], 'BO0 11': ['onwind', 14.4], 'BO0 1': ['onwind', 39.6], 'BO0 19': ['onwind', 50.4]},
    2036: {'BO0 25': ['ror', 11.49], 'BO0 18': ['hydro', 6.23]},
    2037: {'BO0 27': ['ror', 6.81], 'BO0 1': ['OCGT', 57], 'BO0 6': ['OCGT', 1.49 + 1.49 + 1.6]},
    2038: {'BO0 1': ['OCGT', 18.79], 'BO0 6': ['OCGT', 1.55 + 1.51 + 1.6]},
    2039: {'BO0 11': ['biomass', 21], 'BO0 18': ['hydro', 6.2]},
    2040: {'BO0 5': ['OCGT', 26.43 + 25.8 + 26.81 + 26.17], 'BO0 10': ['oil', 1.28]},
}

# Function to determine the period for a given year
def get_period(year):
    return (year - start_year) // LeapYearsInvestment

# Initialize the result dictionary
periodic_decom = {}

# Aggregate decommissioning data into periods
for year, projects in decom.items():
    period = get_period(year)
    if period not in periodic_decom:
        periodic_decom[period] = {}
    for site, data in projects.items():
        technology = data[0]
        capacity = data[1]
        if (site, technology) not in periodic_decom[period]:
            periodic_decom[period][(site, technology)] = 0
        periodic_decom[period][(site, technology)] += capacity


In [16]:
# Helper functions
def get_new_load(dataframe, columns_to_sum):
    for i, col in enumerate(columns_to_sum):
        while col not in dataframe.columns:
            col = str(int(col)-1)
            columns_to_sum[i] = col
    return dataframe[columns_to_sum].sum(axis=1)

def get_new_profile(dataframe, columns_to_average):
    for i, col in enumerate(columns_to_average):
        while col not in dataframe.columns:
            col = str(int(col)-1)
            columns_to_average[i] = col
    return dataframe[columns_to_average].mean(axis=1)

def split_old_buses(old_buses):
    if isinstance(old_buses, str):
        return [bus.strip() for bus in old_buses.split(',')]
    else:
        return [str(old_buses).strip()]  # Handle cases where there's only a single bus name

electricload.csv

In [17]:
electricity_load = pd.read_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv')
pypsa_demand_profiles = pd.read_csv('data_pypsa/demand_profiles.csv')
pypsa_bus_map = pd.read_excel('data_pypsa/bus_map30.xlsx')
bus_numbers_dict = pypsa_bus_map.set_index('Buses')['Gamle noder'].to_dict()

last_three_columns = electricity_load.iloc[:,-4:].copy()
electricity_load = electricity_load.iloc[:,0:0]

for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_load(pypsa_demand_profiles, old_buses_list)
    electricity_load[new_bus] = new_values

electricity_load = pd.concat([electricity_load, last_three_columns.reset_index(drop=True)], axis=1)
electricity_load['time'] = pd.to_datetime(pypsa_demand_profiles['time'])
electricity_load['hour'] = electricity_load['time'].dt.hour
electricity_load['dayofweek'] = electricity_load['time'].dt.dayofweek
electricity_load['month'] = electricity_load['time'].dt.month

# Then change its format to string with a different appearance
electricity_load['time'] = electricity_load['time'].dt.strftime('%d/%m/%Y %H:%M')

electricity_load.to_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv', index=False)

C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3022659369.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  electricity_load = pd.read_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv')


solar.csv

In [18]:
solar = pd.read_csv('Data handler/bolivia_v1/ScenarioData/solar.csv')
weather_path = "data_pypsa/weather_data/"

# Load the solar data
solar_data_11 = xr.open_dataset(weather_path + "profile_solar_2011.nc")
solar_data_13 = xr.open_dataset(weather_path + "profile_solar_2013.nc")
solar_data_18 = xr.open_dataset(weather_path + "profile_solar_2018.nc")
solar_data = xr.concat([solar_data_11, solar_data_13, solar_data_18], dim='time')

# Convert to DataFrame and process
profile_data = solar_data['profile']
profile_df = profile_data.to_dataframe().reset_index()
profile_pivot = profile_df.pivot(index='time', columns='bus', values='profile')
profile_pivot.index = pd.to_datetime(profile_pivot.index)
profile_pivot.reset_index(inplace=True)
profile_pivot.to_csv(weather_path + 'solar_data.csv', index=False)

# Load the newly created CSV
pypsa_solar = pd.read_csv(weather_path + 'solar_data.csv')
solar = pd.DataFrame()
solar['time'] = pd.to_datetime(pypsa_solar['time'],)

# Then change its format to string with a different appearance
solar['time'] = solar['time'].dt.strftime('%d/%m/%Y %H:%M')

# Assuming bus_numbers_dict and split_old_buses, get_new_profile are defined elsewhere
for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_profile(pypsa_solar, old_buses_list)
    solar[new_bus] = new_values

solar.to_csv('Data handler/bolivia_v1/ScenarioData/solar.csv', index=False)

windonshore.csv

In [19]:
wind = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv')

wind_data_11 = xr.open_dataset(weather_path + "profile_onwind_2011.nc")
wind_data_13 = xr.open_dataset(weather_path + "profile_onwind_2013.nc")
wind_data_18 = xr.open_dataset(weather_path + "profile_onwind_2018.nc")

wind_data = xr.concat([wind_data_11, wind_data_13, wind_data_18], dim='time')

profile_data = wind_data['profile']
profile_df = profile_data.to_dataframe().reset_index()
profile_pivot = profile_df.pivot(index='time', columns='bus', values='profile')
profile_pivot.index = pd.to_datetime(profile_pivot.index)
profile_pivot.reset_index(inplace=True)

profile_pivot.to_csv(weather_path + 'wind_data.csv', index=False)

pypsa_wind = pd.read_csv(weather_path + 'wind_data.csv')
wind = wind[['time']].copy()
wind['time'] = pd.to_datetime(pypsa_wind['time'])

# Then change its format to string with a different appearance
wind['time'] = wind['time'].dt.strftime('%d/%m/%Y %H:%M')

for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_profile(pypsa_wind, old_buses_list)
    wind[new_bus] = new_values

wind.to_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv', index=False)

C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\1026634684.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  wind = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv')


In [20]:
windoff = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windoffshore.csv')

windoff = windoff[['time']].copy()
windoff['time'] = pd.to_datetime(pypsa_wind['time'])

# Then change its format to string with a different appearance
windoff['time'] = windoff['time'].dt.strftime('%d/%m/%Y %H:%M')

# for new_bus, old_buses in bus_numbers_dict.items():
#     old_buses_list = split_old_buses(old_buses)
#     windoff[new_bus] = 0

windoff.to_csv('Data handler/bolivia_v1/ScenarioData/windoffshore.csv', index=False)

hydroror.csv & hydroseasonal.csv

In [21]:
# Regulated
hydro_inflows_path = "data_pypsa/inflows_EMPIRE.xlsx"
 
reg_inflows = pd.read_excel(hydro_inflows_path, sheet_name="reg")
 
reg_inflows["time"] = pd.to_datetime(reg_inflows["time"])
reg_inflows["hour"] = reg_inflows["time"].dt.hour
reg_inflows["dayofweek"] = reg_inflows["time"].dt.dayofweek
reg_inflows["month"] = reg_inflows["time"].dt.month
reg_inflows["time"] = reg_inflows["time"].dt.strftime('%d/%m/%Y %H:%M')
                                                 
reg_inflows.to_csv(weather_path + 'hydroseasonal_data.csv', index=False)
reg_inflows.to_csv("Data handler/bolivia_v1/ScenarioData/hydroseasonal.csv", index=False)
 
 
# Run of the river
ror_inflows = pd.read_excel(hydro_inflows_path, sheet_name="ror")
 
ror_inflows["time"] = pd.to_datetime(ror_inflows["time"])
ror_inflows["hour"] = ror_inflows["time"].dt.hour
ror_inflows["dayofweek"] = ror_inflows["time"].dt.dayofweek
ror_inflows["month"] = ror_inflows["time"].dt.month
ror_inflows["time"] = ror_inflows["time"].dt.strftime('%d/%m/%Y %H:%M')
                                                 
ror_inflows.to_csv(weather_path + 'hydroror_data.csv', index=False)
ror_inflows.to_csv("Data handler/bolivia_v1/ScenarioData/hydroror.csv", index=False)
ror_inflows = pd.read_excel(hydro_inflows_path, sheet_name="ror")
 
ror_inflows["time"] = pd.to_datetime(ror_inflows["time"])
ror_inflows["hour"] = ror_inflows["time"].dt.hour
ror_inflows["dayofweek"] = ror_inflows["time"].dt.dayofweek
ror_inflows["month"] = ror_inflows["time"].dt.month
ror_inflows["time"] = ror_inflows["time"].dt.strftime('%d/%m/%Y %H:%M')
                                                 
ror_inflows.to_csv(weather_path + 'hydroror_data.csv', index=False)
ror_inflows.to_csv("Data handler/bolivia_v1/ScenarioData/hydroror.csv", index=False)

General.xlsx

In [22]:
# Load the workbook
workbook = load_workbook('data_pypsa/emissions_annual.xlsx')

# Get the active sheet
sheet = workbook.active

emissions = {}

# Iterate over the rows starting from the second row (skipping the header)
for row in sheet.iter_rows(min_row=2, values_only=True):
    year = row[0]  # Assuming the year is in the first column
    emission_value = row[1]  # Assuming the emission value is in the second column
    emissions[year] = emission_value
emissions

period_ends = [start_year + LeapYearsInvestment + i * LeapYearsInvestment - 1 for i in range((Horizon - start_year) // LeapYearsInvestment)]


In [23]:
workbook = load_workbook('Data handler/bolivia_v1/General.xlsx')

# seasonScale


# CO2Cap
co2_cap_sheet = workbook['CO2Cap']

for i, year in enumerate(period_ends):
    co2_cap_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_cap_sheet.cell(row=i+4, column=2, value=emissions[year])

for i in range(total_periods+4, co2_cap_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_cap_sheet.cell(row=i, column=col).value is not None:
            co2_cap_sheet.cell(row=i, column=col).value = None

# CO2Price
co2_price_sheet = workbook['CO2Price']
for i in range(total_periods):
    co2_price_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_price_sheet.cell(row=i+4, column=2, value=0)

for i in range(total_periods+4, co2_price_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_price_sheet.cell(row=i, column=col).value is not None:
            co2_price_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/General.xlsx')

Generator.xlsx

In [24]:
file_path = 'Data handler/bolivia_v1/Generator.xlsx'
# Load the generator workbook
workbook = load_workbook(file_path)
 
technologies_path = 'data_pypsa/technologies.xlsx'
costs_path = 'data_pypsa/costs_EMPIRE.xlsx'
power_plants_path = 'data_pypsa/custom_powerplants_EMPIRE.xlsx'
generators_path = 'data_pypsa/generators30.xlsx'
generators_empire_path = 'Data handler/europe_v51/Generator.xlsx'
 
# Function to clear all rows below the header in a sheet
def clear_sheet_rows(ws):
    for row in ws.iter_rows(min_row=4):
        for cell in row:
            cell.value = None
            cell.fill = PatternFill(fill_type=None)
           
# Function to clear all columns to the right of the header in a sheet
def clear_sheet_columns(ws):
    for col in ws.iter_cols(min_col=4):
        for cell in col:
            cell.fill = PatternFill(fill_type=None)
 
 
# Load new technologies
new_techs_df = pd.read_excel(technologies_path, sheet_name='technologies_pypsa')
new_technologies = sorted(set(new_techs_df['New_names']))
 
# Load cost data from pypsa
cost_data = pd.read_excel(costs_path, sheet_name='Sheet1')
# Load custom power plant data
power_plants_data = pd.read_excel(power_plants_path, sheet_name='Sheet1')
# Load generator data
generators_data = pd.read_excel(generators_path, sheet_name='Sheet1')
# Load generator rampRate data from empire data
generators_empire_data = pd.read_excel(generators_empire_path, sheet_name='RampRate')
 
# Concatenate the dataframes
tot_gen = pd.concat([generators_data[['bus', 'carrier', 'p_nom']], storage_units[['bus', 'carrier', 'p_nom']]])
 
# Reset the index of the concatenated dataframe
tot_gen = tot_gen.reset_index(drop=True)
tot_gen.sort_values(by=['bus'], inplace=True)
 
# Map new technology names to old names for lookup
tech_name_map = new_techs_df.set_index('New_names')['Old_names'].to_dict()
 
tech_name_map2 = new_techs_df.set_index('New_names')['Fueltype_Technology'].to_dict()
 
# Filter the cost data for investment entries only
investment_cost_data = cost_data[cost_data['parameter'].str.contains('investment', case=False, na=False)]
# Filter the cost data for fixed O&M entries only
FOM_cost_data = cost_data[cost_data['parameter'].str.contains('FOM', case=False, na=False)]
# Filter the cost data for variable O&M entries only
VOM_cost_data = cost_data[cost_data['parameter'].str.contains('VOM', case=False, na=False)]
# Filter the cost data for fuel cost entries only
fuel_cost_data = cost_data[cost_data['parameter'].str.contains('fuel', case=False, na=False)]
# Filter the cost data for CO2 intensity entries only
co2_intensity_data = cost_data[cost_data['parameter'].str.contains('CO2 intensity', case=False, na=False)]
# Filter the cost data for lifetime entries only
lifetime_data = cost_data[cost_data['parameter'].str.contains('lifetime', case=False, na=False)]
 
# Filter the cost data for efficiency entries only
efficiency_data = cost_data[cost_data['parameter'].str.contains('efficiency', case=False, na=False)]
 
# Dictionary to hold the investment values for each new technology
investment_costs = {}
FOM = {}
VOM = {}
fuel = {}
efficiencies = {}
co2_intensities = {}
lifetimes = {}
 
 
 
for new_tech, old_tech in tech_name_map.items():
    tech_cost_data = investment_cost_data[investment_cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_cost_data.empty:
        investment_costs[new_tech] = tech_cost_data.iloc[0]['value']
    else:
        investment_costs[new_tech] = 0
       
    tech_FOM = FOM_cost_data[FOM_cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_FOM.empty:
        FOM[new_tech] = tech_FOM.iloc[0]['value']/100 * investment_costs[new_tech]
    else:
        FOM[new_tech] = 0
 
    tech_VOM = VOM_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_VOM.empty:
        VOM[new_tech] = tech_VOM.iloc[0]['value']
    else:
        VOM[new_tech] = 0
       
    tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
 
    if new_tech == 'Gas OCGT' or new_tech == 'Gas CCGT':
        tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == 'gas']
    if not tech_fuel.empty:
        # Change the fuel cost from EUR/MWh to EUR/GJ
        fuel[new_tech] = tech_fuel.iloc[0]['value'] / 3.6
    else:
        fuel[new_tech] = 0
       
    tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if new_tech == 'Gas OCGT' or new_tech == 'Gas CCGT':
         tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == 'gas']
    if not tech_co2.empty:
        co2_intensities[new_tech] = tech_co2.iloc[0]['value'] / 3.6
    else:
        co2_intensities[new_tech] = 0
       
    tech_lifetime = lifetime_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_lifetime.empty:
        lifetimes[new_tech] = tech_lifetime.iloc[0]['value']
    else:
        lifetimes[new_tech] = 0
       
    tech_efficiency = efficiency_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_efficiency.empty:
        efficiencies[new_tech] = tech_efficiency.iloc[0]['value']
    else:
        efficiencies[new_tech] = 1
   
 
for sheet in workbook.sheetnames:
    if sheet in ["CapitalCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,total_periods+1):
            for i, (tech, cost) in enumerate(investment_costs.items()):
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=1, value=tech)
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=2, value=period)
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=3, value=cost)
             
               
    elif sheet in ["FixedOMCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,total_periods+1):
            for i, (tech, cost) in enumerate(FOM.items()):
                ws.cell(row=4+(period-1) * len(FOM) + i, column=1, value=tech)
                ws.cell(row=4+(period-1) * len(FOM) + i, column=2, value=period)
                ws.cell(row=4+(period-1) * len(FOM) + i, column=3, value=round(cost,3))
               
               
    elif sheet in ["VariableOMCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, cost) in enumerate(VOM.items()):
                ws.cell(row=4+i, column=1, value=tech)
                ws.cell(row=4+i, column=2, value=cost)
   
    elif sheet in ["FuelCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,total_periods+1):
            for i, (tech, cost) in enumerate(fuel.items()):
                ws.cell(row=4+(period-1) * len(fuel) +i, column=1, value=tech)
                ws.cell(row=4+(period-1) * len(fuel) +i, column=2, value=period)
                ws.cell(row=4+(period-1) * len(fuel) +i, column=3, value=cost)
               
        if pathway == "GC":
            for index, row in enumerate(ws.iter_rows(), start=1):
                cell_value = row[0].value  # Assuming column 1 is at index 0
                if cell_value in ["Gas OCGT", "Gas CCGT", "Oil existing"]:
                    period = row[1].value  # Assuming period is in column 2, at index 1
                    tech = cell_value
                    #Now use 'index' as the row parameter to set the value
                    ws.cell(row=index, column=3, value=gc_fuel_costs[period][tech])
 
    elif sheet in ["CCSCostTSVariable"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        clear_sheet_columns(ws)
       
        row_counter = 4
       
        for period in range(1,total_periods+1):
            ws.cell(row=row_counter, column=1, value=period)
            ws.cell(row=row_counter, column=2, value=0)
            row_counter += 1
               
       
    elif sheet in ["Efficiency"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        row = 4
        for i, (tech, data) in enumerate(efficiencies.items()):
            for period in range(1,total_periods+1):
                ws.cell(row=row, column=1, value=tech)
                ws.cell(row=row, column=2, value=period)
                ws.cell(row=row, column=3, value=data)
                row += 1
               
    elif sheet in ["RefInitialCap"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        clear_sheet_columns(ws)
       
        row_counter = 4
       
        for i, bus in tot_gen['bus'].items():
            inv_tech_name_map = {v: k for k, v in tech_name_map.items()}
            old_carrier_name = inv_tech_name_map.get(tot_gen['carrier'][i], tot_gen['carrier'][i])
            if tot_gen['carrier'][i] != 'load':
                if bus == "BO0 5" and ws.cell(row=row_counter-1, column=1).value == "BO0 4":
                    ws.cell(row=row_counter, column=1, value=bus)
                    ws.cell(row=row_counter, column=2, value="Hydro run-of-the-river")
                    ws.cell(row=row_counter, column=3, value=0)  
                    row_counter += 1
                ws.cell(row=row_counter, column=1, value=bus)
                ws.cell(row=row_counter, column=2, value=old_carrier_name)
                ws.cell(row=row_counter, column=3, value=0)
               
                row_counter += 1
   
    elif sheet in ["ScaleFactorInitialCap"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(2,total_periods+1):
            for i, tech in enumerate(new_technologies):
                ws.cell(row=4+(period-2)*8+i, column=1, value=tech)
                ws.cell(row=4+(period-2)*8+i, column=2, value=period)
                ws.cell(row=4+(period-2)*8+i, column=3, value=0)
               
    elif sheet in ["InitialCapacity"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
       
        row_counter = 4
       
        inv_tech_name_map = {v: k for k, v in tech_name_map.items()}
 
        # Initialize adjustment tracking
        adjustments = {}
 
        # Loop over periods and entries
        for period in range(1, total_periods + 1):
            for i, row in tot_gen.iterrows():
                bus = row['bus']
                carrier = row['carrier']
                p_nom = row['p_nom']
               
                # Calculate the generator's adjusted nominal power
                key = (bus, carrier)
           
                if key in periodic_decom.get(period, {}):
                    adjustments[key] = adjustments.get(key, 0) + periodic_decom[period][key]
                   
                p_nom -= adjustments.get(key, 0)
               
                # Skip load type carriers
                if carrier != 'load':
                    if bus == "BO0 5" and ws.cell(row=row_counter-1, column=1).value == "BO0 4":
                        ws.cell(row=row_counter, column=1, value=bus)
                        ws.cell(row=row_counter, column=2, value="Hydro run-of-the-river")
                        ws.cell(row=row_counter, column=3, value=period)
                        ws.cell(row=row_counter, column=4, value=0)  
                        row_counter += 1
                    ws.cell(row=row_counter, column=1, value=bus)
                    ws.cell(row=row_counter, column=2, value=inv_tech_name_map.get(carrier, carrier))
                    ws.cell(row=row_counter, column=3, value=period)
                    ws.cell(row=row_counter, column=4, value=p_nom)
 
                    row_counter += 1
                   
            print(period, adjustments)
           
       
    elif sheet in ["MaxBuiltCapacity"]:  ## Usikker her på hva som skal inn, i EMPIREs data er det kun 0 som verdier
        ws = workbook[sheet]
        clear_sheet_rows(ws)
       
        row_counter = 4
       
        for period in range(1,total_periods+1):
            for j, tech in enumerate(new_technologies):
                tech_max_installed_capacity = {"Geo" : 510, "Bio" : 840}
                for i, bus in generators_data['bus'].items():
                    if tech_name_map[tech] == generators_data['carrier'][i]:
                        ws.cell(row=row_counter, column=1, value=bus)
                        ws.cell(row=row_counter, column=2, value=generator_tech_dict[tech][0])
                        ws.cell(row=row_counter, column=3, value=period)        
                        if tech in tech_max_installed_capacity:
                            ws.cell(row=row_counter, column=4, value=tech_max_installed_capacity[tech])
                        elif tech == 'Hydro regulated':
                            ws.cell(row=row_counter, column=4, value=0)
                        elif tech == 'Hydro run-of-the-river':
                            ws.cell(row=row_counter, column=4, value=500000)
                            if ws.cell(row=row_counter-1, column=1).value == "BO0 0":
                                row_counter += 1
                                ws.cell(row=row_counter, column=1, value='BO0 5')
                                ws.cell(row=row_counter, column=2, value=generator_tech_dict[tech][0])
                                ws.cell(row=row_counter, column=3, value= period)
                                ws.cell(row=row_counter, column=4, value= 89.73)
                            if bus == 'BO0 18':
                                ws.cell(row=row_counter, column=4, value=118 + 85)
                        else:
                            ws.cell(row=row_counter, column=4, value=500000)
                       
                           
                        row_counter += 1
               
    elif sheet in ["MaxInstalledCapacity"]:  
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        clear_sheet_columns(ws)
       
        row_counter = 4
       
        for j, tech in enumerate(new_technologies):
            tech_max_installed_capacity = {"Geo" : 510, "Bio" : 840}
            for i, bus in generators_data['bus'].items():
                if tech_name_map[tech] == generators_data['carrier'][i]:
                    ws.cell(row=row_counter, column=1, value=bus)
                    ws.cell(row=row_counter, column=2, value=generator_tech_dict[tech][0])
                    if tech in tech_max_installed_capacity:
                        ws.cell(row=row_counter, column=3, value=tech_max_installed_capacity[tech])
                    elif tech == 'Hydro regulated':
                        ws.cell(row=row_counter, column=3, value=generators_data['p_nom'][i])
                    elif tech == 'Hydro run-of-the-river':
                        ws.cell(row=row_counter, column=3, value=generators_data['p_nom'][i])
                        if bus == 'BO0 0':
                            row_counter += 1
                            ws.cell(row=row_counter, column=1, value='BO0 5')
                            ws.cell(row=row_counter, column=2, value=generator_tech_dict[tech][0])
                            ws.cell(row=row_counter, column=3, value= 89.73)
                        if bus == 'BO0 18':
                            ws.cell(row=row_counter, column=3, value=generators_data['p_nom'][i] + 85 + 118)
                    else:
                        ws.cell(row=row_counter, column=3, value=100000)
                    row_counter += 1
           
               
   
    elif sheet in ["RampRate"]: ## HARD CODED
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        thermal_gens = ['Gas OCGT', 'Bio', 'Gas CCGT', 'Oil existing', 'Geo']
        if val:
            ramprates = {'Gas OCGT' : 1.00, 'Bio' : 1.00, 'Gas CCGT' : 1.00, 'Oil existing' : 1.00, 'Geo' : 1.00}
        if not val:
            ramprates = {'Gas OCGT' : 1.00, 'Bio' : 0.70, 'Gas CCGT' : 0.85, 'Oil existing' : 0.85, 'Geo' : 1.00}
        for i, (tech, rate) in enumerate(ramprates.items()):
            ws.cell(row=4+i, column=1, value=tech)  
            ws.cell(row=4+i, column=2, value=rate)
           
    elif sheet in ["GeneratorTypeAvailability"]: ## HARD CODED
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        availabilities = {'Gas CCGT' : 1, 'Geo' : 1, 'Wind onshore' : 0, 'Bio' : 1, 'Solar' : 0, 'Gas OCGT' : 1, 'Oil existing' : 1, 'Hydro run-of-the-river' : 0, 'Hydro regulated' : 1}
        for i, (tech, avail) in enumerate(availabilities.items()):
            ws.cell(row=4+i, column=1, value=tech)  
            ws.cell(row=4+i, column=2, value=avail)
               
    elif sheet in ["CO2Content"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, co2) in enumerate(co2_intensities.items()):
            ws.cell(row=4+i, column=1, value=tech)
            ws.cell(row=4+i, column=2, value=co2)    
               
           
    elif sheet in ["Lifetime"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, lifetime) in enumerate(lifetimes.items()):
            ws.cell(row=4+i, column=1, value=tech)
            ws.cell(row=4+i, column=2, value=lifetime)
     
workbook.save(file_path)
 
print(adjustments)
print(periodic_decom)

C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3637883313.py:90: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_VOM = VOM_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3637883313.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3637883313.py:106: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3637883313.py:114: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_lifetime = lifetime_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\noraky\AppData\Local\Temp\ipykernel_40636\3637883313.py:120: 

1 {('BO0 1', 'OCGT'): 18.1, ('BO0 10', 'oil'): 3.3400000000000003, ('BO0 18', 'hydro'): 2.55, ('BO0 5', 'OCGT'): 83.56}
2 {('BO0 1', 'OCGT'): 93.88999999999999, ('BO0 10', 'oil'): 4.44, ('BO0 18', 'hydro'): 14.98, ('BO0 5', 'OCGT'): 83.56, ('BO0 1', 'onwind'): 39.6, ('BO0 11', 'onwind'): 14.4, ('BO0 11', 'biomass'): 21, ('BO0 19', 'onwind'): 50.4, ('BO0 2', 'solar'): 5, ('BO0 25', 'ror'): 11.49, ('BO0 27', 'ror'): 6.81, ('BO0 28', 'oil'): 1.1, ('BO0 6', 'OCGT'): 9.24}
3 {('BO0 1', 'OCGT'): 93.88999999999999, ('BO0 10', 'oil'): 5.720000000000001, ('BO0 18', 'hydro'): 14.98, ('BO0 5', 'OCGT'): 188.77, ('BO0 1', 'onwind'): 39.6, ('BO0 11', 'onwind'): 14.4, ('BO0 11', 'biomass'): 21, ('BO0 19', 'onwind'): 50.4, ('BO0 2', 'solar'): 5, ('BO0 25', 'ror'): 11.49, ('BO0 27', 'ror'): 6.81, ('BO0 28', 'oil'): 1.1, ('BO0 6', 'OCGT'): 9.24}
{('BO0 1', 'OCGT'): 93.88999999999999, ('BO0 10', 'oil'): 5.720000000000001, ('BO0 18', 'hydro'): 14.98, ('BO0 5', 'OCGT'): 188.77, ('BO0 1', 'onwind'): 39.6, (

In [25]:
# Load the workbook with production data
inflows_workbook = load_workbook('data_pypsa/inflows_EMPIRE.xlsx')
inflows_sheet_reg = inflows_workbook["reg"]
inflows_sheet_ror = inflows_workbook["ror"]

# Dictionary to store total production for each bus
production_totals_reg = {}

# Assume each column after the first in the first row has bus names and subsequent rows have hourly production values
for col in range(2, inflows_sheet_reg.max_column + 1):
    bus_name = inflows_sheet_reg.cell(row=1, column=col).value
    total_production = sum(inflows_sheet_reg.cell(row=row, column=col).value for row in range(2, inflows_sheet_reg.max_row + 1))
    production_totals_reg[bus_name] = total_production
    
production_totals_ror = {}

for col in range(2, inflows_sheet_ror.max_column + 1):
    bus_name = inflows_sheet_ror.cell(row=1, column=col).value
    total_production = sum(inflows_sheet_ror.cell(row=row, column=col).value*tot_gen[(tot_gen['carrier'] == 'ror') & (tot_gen['bus'] == bus_name)]['p_nom'].sum()for row in range(2, inflows_sheet_ror.max_row + 1))
    production_totals_ror[bus_name] = total_production


Node.xlsx

In [26]:
import functools
import operator

pypsa_load = load_workbook('data_pypsa/load_per_gen30.xlsx').active
load_dict = {row[0]: row[1] for row in pypsa_load.iter_rows(min_row=2, values_only=True)}


# Function to calculate cumulative scaling factor
def calculate_cumulative_scaling(start_year, end_year, scale_demand):
    return functools.reduce(operator.mul, (scale_demand[year] for year in range(start_year, end_year + 1)), 1)

# Calculate the demands for the last year of each period
period_ends = [start_year + LeapYearsInvestment + i * LeapYearsInvestment - 1 for i in range((Horizon - start_year) // LeapYearsInvestment)]
print(period_ends)
demands = {}
for index, year in enumerate(period_ends):
    period_number = index + 1  # Period number, starting from 1
   
    if year in scale_demand:
        cumulative_scaling_factor = calculate_cumulative_scaling(start_year, year, scale_demand)
        demands[period_number] = {node: load * cumulative_scaling_factor for node, load in load_dict.items()}


[2029, 2034, 2039]


In [27]:

workbook = load_workbook('Data handler/bolivia_v1/Node.xlsx')

for sheet in workbook.sheetnames:
    current_sheet = workbook[sheet]

    nodes_positions = {'A3': ('B3', 1), 'E3': ('F3', 5)}
    
    for node_cell_ref, (period_cell_ref, col_index) in nodes_positions.items():
        if current_sheet[node_cell_ref].value == 'Nodes':
            # Check if the cell next to "Nodes" contains "Period"
            if current_sheet[period_cell_ref].value == 'Period':
                # Logic for handling periods
                start_row = current_sheet[node_cell_ref].row + 1
                
                # Repeat names for each period
                for period_index in range(total_periods):
                    period_start_row = start_row + (period_index * len(new_names))
                    
                    # Fill names for the current period
                    for i, name in enumerate(new_names, start=period_start_row):
                        current_sheet.cell(row=i, column=col_index, value=name)
                        current_sheet.cell(row=i, column=col_index + 1, value=period_index + 1)
                        if sheet == 'ElectricAnnualDemand':
                            current_sheet.cell(row=i, column=col_index + 2, value=demands[period_index+1][name])
                            
                        elif sheet == 'NodeLostLoadCost':
                            current_sheet.cell(row=i, column=col_index + 2, value=8000000) # Cost as in PyPSA

                # Clear cells after the last period's names, if necessary
                last_name_row = start_row + (total_periods * len(new_names))
                for i in range(last_name_row, current_sheet.max_row + 1):
                    for col in range(1, 8): 
                        if current_sheet.cell(row=i, column=col).value is not None:
                            current_sheet.cell(row=i, column=col).value = None

            else:
                # Max annual production for hydro
                clear_sheet_rows(current_sheet)
                # Fill names directly if no "Period" is next to "Nodes"
                row_counter = 4
                for i, name in enumerate(production_totals_reg.keys() | production_totals_ror.keys()):
                    current_sheet.cell(row=row_counter, column=1, value=name)
                    if name in production_totals_reg and name in production_totals_ror:
                        current_sheet.cell(row=row_counter, column=2, value=production_totals_reg[name] + production_totals_ror[name])
                    elif name in production_totals_reg and name not in production_totals_ror:
                        current_sheet.cell(row=row_counter, column=2, value=production_totals_reg[name])
                    else:
                        current_sheet.cell(row=row_counter, column=2, value=production_totals_ror[name])
                        
                    row_counter += 1

workbook.save('Data handler/bolivia_v1/Node.xlsx')

Sets.xlsx

In [28]:
workbook = load_workbook('Data handler/bolivia_v1/Sets.xlsx')

# Nodes
node_sheet = workbook['Nodes']

max_row = node_sheet['A1'].row + len(new_names)
# Fill names directly if no "Period" is next to "Nodes"
for i, name in enumerate(new_names, start=node_sheet['A1'].row + 1):
    node_sheet.cell(row=i, column=1, value=name)

for i in range(max_row + 1, node_sheet.max_row + 1):
    if node_sheet.cell(row=i, column=1).value is not None:
        node_sheet.cell(row=i, column=1).value = None


# OffshoreNodes
offshore_node_sheet = workbook['OffshoreNodes']
max_row = offshore_node_sheet.max_row

for i in range(1, max_row):
    if offshore_node_sheet.cell(row=i + 1, column=1).value is not None:
        offshore_node_sheet.cell(row=i + 1, column=1).value = None


# Horizon
for i in range(1, total_periods + 1):
    workbook['Horizon'].cell(row=i + 1, column=1, value=i)

for i in range(total_periods + 2, workbook['Horizon'].max_row + 1):
    if workbook['Horizon'].cell(row=i, column=1).value is not None:
        workbook['Horizon'].cell(row=i, column=1).value = None

# Storage
storage_sheet = workbook['Storage']
storages = ['Li-Ion_BESS']
for i in range(len(storages)):
    storage_sheet.cell(row=2+i, column=1, value=storages[i])

storage_sheet.cell(row=2+len(storages), column=1).value=None

# Technology
techs = ['Existing', 'Gas', 'Oil', 'Bio', 'Geo', 'Hydro_reg', 'Hydro_ror', 'Wind_onshr', 'Solar']
tech_sheet = workbook['Technology']
clear_sheet_rows(tech_sheet)
for i in range(len(techs)):
    tech_sheet.cell(row=i + 2, column=1, value=techs[i])

# Generators
generators_sheet = workbook['Generators']
for index, row in enumerate(pypsa_tech.iter_rows(min_row=2, values_only=True), start=1):
    generators_sheet.cell(row=index + 1, column=1, value=row[2])

max_row = 1 + len(pypsa_tech['C'])
for i in range(max_row, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=1).value is not None:
        generators_sheet.cell(row=i, column=1).value = None

thermal_gens = ['Gas OCGT', 'Bio', 'Gas CCGT', 'Oil existing', 'Geo']
for i in range(len(thermal_gens)):
    generators_sheet.cell(row=i + 2, column=4, value=thermal_gens[i])

max_row = 1 + len(thermal_gens)
for i in range(max_row + 1, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=4).value is not None:
        generators_sheet.cell(row=i, column=4).value = None

# LineType
line_sheet = workbook['LineType']
line_sheet.cell(row=2, column=1, value='HVAC_OverheadLine')
line_sheet.cell(row=3, column=1).value=None

# HourOfSeason
# Not in use

# StorageOfNodes
storage_node_sheet = workbook['StorageOfNodes']
max_row = storage_node_sheet['A3'].row + len(new_names)*len(storages)

row_index = 4
for name in new_names:    
    for storage in storages:
        storage_node_sheet.cell(row=row_index, column=1, value=name)
        storage_node_sheet.cell(row=row_index, column=2, value=storage)
        row_index += 1

for i in range(max_row + 1, storage_node_sheet.max_row + 1):
    for col in range(1, 3): 
        if storage_node_sheet.cell(row=i, column=col).value is not None:
            storage_node_sheet.cell(row=i, column=col).value = None

# DirectionalLines
line_connections = workbook['DirectionalLines']
pypsa_connections = load_workbook('data_pypsa/lines30.xlsx').active
row_index = 4
for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_connections.cell(row=row_index, column=1, value=value_a)
    line_connections.cell(row=row_index, column=2, value=value_b)
    row_index += 1
    line_connections.cell(row=row_index, column=1, value=value_b)
    line_connections.cell(row=row_index, column=2, value=value_a)
    row_index += 1

max_row = 2 + len(pypsa_connections['A'])*2
for i in range(max_row + 1, line_connections.max_row + 1):
    for col in range(1, 4): 
        if line_connections.cell(row=i, column=col).value is not None:
            line_connections.cell(row=i, column=col).value = None

# LineTypeOfDirectionalLines
line_type_connections = workbook['LineTypeOfDirectionalLines']

row_index = 4
for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_type_connections.cell(row=row_index, column=1, value=value_a)
    line_type_connections.cell(row=row_index, column=2, value=value_b)
    line_type_connections.cell(row=row_index, column=3, value='HVAC_OverheadLine')

    row_index += 1
    line_type_connections.cell(row=row_index, column=1, value=value_b)
    line_type_connections.cell(row=row_index, column=2, value=value_a)
    line_type_connections.cell(row=row_index, column=3, value='HVAC_OverheadLine')

    row_index += 1

for i in range(max_row + 1, line_type_connections.max_row + 1):
    for col in range(1, 4): 
        if line_type_connections.cell(row=i, column=col).value is not None:
            line_type_connections.cell(row=i, column=col).value = None


# GeneratorsOfNode
generators_node_sheet = workbook['GeneratorsOfNode']
ws = generators_node_sheet
clear_sheet_rows(ws)
       
row_counter = 4
        
inv_tech_name_map = {v: k for k, v in tech_name_map.items()}
# Initialize adjustment tracking
adjustments = {}

# Loop over periods and entries
for i, row in tot_gen.iterrows():
    bus = row['bus']
    carrier = row['carrier']
    p_nom = row['p_nom']
        
    # Skip load type carriers
    if carrier != 'load':
        if bus == "BO0 5" and ws.cell(row=row_counter-1, column=1).value == "BO0 4":
                        ws.cell(row=row_counter, column=1, value=bus)
                        ws.cell(row=row_counter, column=2, value="Hydro run-of-the-river")
                        row_counter += 1
        ws.cell(row=row_counter, column=1, value=bus)
        ws.cell(row=row_counter, column=2, value=inv_tech_name_map.get(carrier, carrier))
        
        row_counter += 1
    
    

# GeneratorsOfTechnology
generator_tech_sheet = workbook['GeneratorsOfTechnology']
clear_sheet_rows(generator_tech_sheet)
row_num = 4
for key, items in generator_tech_dict.items():
    for item in items:
        generator_tech_sheet.cell(row=row_num, column=2, value=key)
        generator_tech_sheet.cell(row=row_num, column=1, value=item)
        row_num += 1

    


# Coords
coord_sheet = workbook['Coords']
pypsa_coords = load_workbook('data_pypsa/buses_EMPIRE.xlsx').active

for index, row in enumerate(pypsa_coords.iter_rows(min_row=2, values_only=True), start=1):
    node, x, y = row[0], row[3], row[4]
    coord_sheet.cell(row=index + 3, column=1, value=node)
    coord_sheet.cell(row=index + 3, column=2, value=y)
    coord_sheet.cell(row=index + 3, column=3, value=x)

for i in range(4 + len(new_names), coord_sheet.max_row + 1):
    for col in range(1, 4): 
        if coord_sheet.cell(row=i, column=col).value is not None:
            coord_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/Sets.xlsx')

Storage.xlsx

In [29]:
workbook = load_workbook('Data handler/bolivia_v1/Storage.xlsx')
# InitialPowerCapacity, PowerMaxBuiltCapacity, EnergyInitialCapacity, EnergyMaxBuiltCapacity, EnergyMaxInstalledCapacity, PowerMaxInstalledCapacity

similar_sheets = ['InitialPowerCapacity', 'PowerMaxBuiltCapacity', 'EnergyInitialCapacity', 'EnergyMaxBuiltCapacity', 'EnergyMaxInstalledCapacity', 'PowerMaxInstalledCapacity']

for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]

    nodes_cell, period_cell = 'A3', 'C3'
    start_row = current_sheet[nodes_cell].row + 1

    if sheet_name == 'InitialPowerCapacity' or sheet_name ==  'EnergyInitialCapacity':        
        for period_index in range(total_periods):
            period_start_row = start_row + (period_index * len(new_names))
            
            # Fill names for the current period
            for i, name in enumerate(new_names, start=period_start_row):
                current_sheet.cell(row=i, column=1, value=name)
                current_sheet.cell(row=i, column=3, value=period_index + 1)
                current_sheet.cell(row=i, column=4, value=0)

            # Clear cells after the last period's names, if necessary
        last_name_row = start_row 
        #last_name_row = start_row + (total_periods * len(new_names))
        for i in range(last_name_row, current_sheet.max_row + 1):
            for col in range(1, 5): 
                if current_sheet.cell(row=i, column=col).value is not None:
                        current_sheet.cell(row=i, column=col).value = None

    else:
        row_index = 4
        for storage in storages:
            for name in new_names:
                current_sheet.cell(row=row_index, column=1, value=name)
                current_sheet.cell(row=row_index, column=2, value=storage)
                if sheet_name == 'PowerMaxBuiltCapacity' or sheet_name == 'EnergyMaxBuiltCapacity':
                    current_sheet.cell(row=row_index, column=3, value=1)
                elif sheet_name == 'EnergyMaxInstalledCapacity':
                    current_sheet.cell(row=row_index, column=3, value=5400)
                elif sheet_name == 'PowerMaxInstalledCapacity':
                    current_sheet.cell(row=row_index, column=3, value=900)
                row_index += 1
    
        last_name_row = start_row + (len(storages) * len(new_names))
        for i in range(last_name_row, current_sheet.max_row + 1):
            for col in range(1, 5): 
                if current_sheet.cell(row=i, column=col).value is not None:
                        current_sheet.cell(row=i, column=col).value = None


   
# PowerCapitalCost, PowerFixedOMCost, EnergyCapitalCost, EnergyFixedOMCost
similar_sheets = ['PowerCapitalCost', 'PowerFixedOMCost', 'EnergyCapitalCost', 'EnergyFixedOMCost']
for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]
    index = 4
    for storage in storages:
        for period in range(1, total_periods + 1):
            current_sheet.cell(row=index, column=1, value=storage)
            current_sheet.cell(row=index, column=2, value=period)
            if sheet_name == 'PowerCapitalCost':
                current_sheet.cell(row=index, column=3, value=investment_cost_data[investment_cost_data['technology'] == 'battery inverter'].iloc[0]['value']*usd_to_eur)
            elif sheet_name == 'EnergyCapitalCost':
                current_sheet.cell(row=index, column=3, value=investment_cost_data[investment_cost_data['technology'] == 'battery storage'].iloc[0]['value']*usd_to_eur)
            index += 1

    for i in range(len(storages)*total_periods+4, current_sheet.max_row + 1):
        for col in range(1, 4): 
            if current_sheet.cell(row=i, column=col).value is not None:
                    current_sheet.cell(row=i, column=col).value = None
    

# StorageInitialEnergyLevel, StorageChargeEff, StorageDischargeEff, StoragePowToEnergy, StorageBleedEfficiency, Lifetime
similar_sheets = ['StorageInitialEnergyLevel', 'StorageChargeEff', 'StorageDischargeEff', 'StorageBleedEfficiency', 'Lifetime']
for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]
    index = 4
    for storage in storages:
        current_sheet.cell(row=index, column=1, value=storage)
        if sheet_name == 'Lifetime':
             current_sheet.cell(row=index, column=2, value=lifetime_data[lifetime_data['technology'] == 'battery storage'].iloc[0]['value'])
        # TODO: Implement values for column 2 in the different sheets

    for i in range(index+len(storages), current_sheet.max_row + 1):
        for col in range(1, 4): 
            if current_sheet.cell(row=i, column=col).value is not None:
                    current_sheet.cell(row=i, column=col).value = None

# StoragePowToEnergy
current_sheet = workbook['StoragePowToEnergy']
index = 4
storage = storages[0]
current_sheet.cell(row=index, column=1, value=storage)
# TODO: Implement values for column 2 in the different sheets

for i in range(6, current_sheet.max_row + 1):
    for col in range(1, 4): 
        if current_sheet.cell(row=i, column=col).value is not None:
                current_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/Storage.xlsx')

Transmission.xlsx

In [30]:
workbook = load_workbook('Data handler/bolivia_v1/Transmission.xlsx')
pypsa_cost = load_workbook('data_pypsa/costs_EMPIRE.xlsx').active
 
# lineEfficiency
current_sheet = workbook['lineEfficiency']
row_offset = 4
for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    node_1, node_2, distance = row[1], row[2], row[3]
    for i in range(0,total_periods):
        current_sheet.cell(row=row_offset, column=1, value=node_1)
        current_sheet.cell(row=row_offset, column=2, value=node_2)
        current_sheet.cell(row=row_offset, column=3, value=1)
        row_offset += 1
    for i in range(0, total_periods):
        current_sheet.cell(row=row_offset, column=1, value=node_2)
        current_sheet.cell(row=row_offset, column=2, value=node_1)
        current_sheet.cell(row=row_offset, column=3, value=1)
        row_offset += 1
 
 
# MaxBuiltCapacity, Length, InitialCapacity, MaxInstallCapacityRaw, Lifetime
similar_sheets = ['MaxBuiltCapacity', 'Length', 'InitialCapacity', 'MaxInstallCapacityRaw', 'Lifetime']
for sheet in similar_sheets:
    current_sheet = workbook[sheet]
    clear_sheet_rows(current_sheet)
    row_offset = 4
    for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
        node_1, node_2, distance, cap = row[1], row[2], row[3], row[8]
       
        if sheet == 'InitialCapacity' or sheet == 'MaxInstallCapacityRaw':
            for i in range(0, total_periods):
                current_sheet.cell(row=row_offset, column=1, value=node_1)
                current_sheet.cell(row=row_offset, column=2, value=node_2)
                current_sheet.cell(row=row_offset, column=3, value=i+1)
                current_sheet.cell(row=row_offset, column=4, value=1000000)
 
                if sheet == 'InitialCapacity':
                    current_sheet.cell(row=row_offset, column=4, value=cap)
                #else:
                    # TODO: Add values for columns 3 in MaxInstallCapacityRaw
                row_offset += 1
   
 
        else:    
            current_sheet.cell(row=index + 3, column=1, value=node_1)
            current_sheet.cell(row=index + 3, column=2, value=node_2)
            if sheet == 'MaxBuiltCapacity':
                current_sheet.cell(row=index + 3, column=3, value=1) # Period
                current_sheet.cell(row=index + 3, column=4, value=20000) # Max investement in transmission line, defailt 20 000 MW in Empire
            elif sheet == 'Length':
                current_sheet.cell(row=index + 3, column=3, value=distance) # Distance between nodes
            elif sheet == 'Lifetime':
                    current_sheet.cell(row=index + 3, column=3, value=pypsa_cost['E185'].value) # Lifetime of transmission line
 
    # if sheet == 'InitialCapacity' or sheet == 'MaxInstallCapacityRaw':
    #     for i in range((len(pypsa_connections['A']))*total_periods + 1, current_sheet.max_row + 1):
    #             for col in range(1, 5):
    #                 if current_sheet.cell(row=i, column=col).value is not None:
    #                     current_sheet.cell(row=i, column=col).value = None    
    # else:
    #     for i in range(3 + len(pypsa_connections['A']), 200):
    #         for col in range(1, 5):
    #             if current_sheet.cell(row=i, column=col).value is not None:
    #                 current_sheet.cell(row=i, column=col).value = None

# TypeCapitalCost
cost_sheets = ['TypeCapitalCost', 'TypeFixedOMCost']
for sheet in cost_sheets:
    current_sheet = workbook[sheet]
    clear_sheet_rows(current_sheet)
    for i in range(1,total_periods+1):
        current_sheet.cell(row=i+3, column=1, value="HVAC_OverheadLine")
        current_sheet.cell(row=i+3, column=2, value=i)
        if sheet == 'TypeCapitalCost':
            current_sheet.cell(row=i+3, column=3, value= pypsa_cost['E184'].value)
        elif sheet == 'TypeFixedOMCost':
            current_sheet.cell(row=i+3, column=3, value= pypsa_cost['E186'].value)
 
    for i in range(total_periods + 5, current_sheet.max_row + 1):
        for col in range(1, 4):
            if current_sheet.cell(row=i, column=col).value is not None:
                current_sheet.cell(row=i, column=col).value = None
 
 
 
workbook.save('Data handler/bolivia_v1/Transmission.xlsx')